## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,59.09,83,40,3.44,scattered clouds
1,1,Avarua,CK,-21.2078,-159.7750,78.85,73,40,13.80,scattered clouds
2,2,Butaritari,KI,3.0707,172.7902,81.70,71,13,11.39,few clouds
3,3,Dikson,RU,73.5069,80.5464,32.50,79,100,16.87,overcast clouds
4,4,Sur,OM,22.5667,59.5289,83.14,65,6,8.95,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                258
City                   258
Country                254
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df_clean = preferred_cities_df.dropna()
preferred_cities_df_clean.count()

City_ID                254
City                   254
Country                254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,59.09,scattered clouds,-33.9258,18.4232,
6,Aleksandrov Gay,RU,51.30,overcast clouds,50.1470,48.5704,
8,Bluff,NZ,54.91,broken clouds,-46.6000,168.3333,
11,Albany,US,62.37,overcast clouds,42.6001,-73.9662,
12,Carnarvon,AU,64.26,clear sky,-24.8667,113.6333,
14,Lusambo,CD,69.22,overcast clouds,-4.9667,23.4500,
20,Rikitea,PF,68.86,clear sky,-23.1203,-134.9692,
22,Bethel,US,66.54,clear sky,41.3712,-73.4140,
26,Kaitangata,NZ,61.12,broken clouds,-46.2817,169.8464,
29,Torbay,CA,53.28,clear sky,47.6666,-52.7314,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except IndexError:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace('', np.nan)
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,59.09,scattered clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
6,Aleksandrov Gay,RU,51.30,overcast clouds,50.1470,48.5704,Gostinitsa
8,Bluff,NZ,54.91,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
12,Carnarvon,AU,64.26,clear sky,-24.8667,113.6333,Hospitality Carnarvon
20,Rikitea,PF,68.86,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
708,Vila Franca Do Campo,PT,67.19,light rain,37.7167,-25.4333,Pestana Bahia Praia
709,Hualmay,PE,61.59,scattered clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
710,Altay,CN,58.46,clear sky,47.8667,88.1167,Jinqiao Hotel
714,Gambela,ET,69.60,light rain,8.2500,34.5833,Solomon Debebe Hotel


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))